## Bibliotecas

In [ ]:
import pandas as pd 
import requests 
from plyer import notification
from datetime import datetime
import notificacao
import sqlite3
import re
import funcoes as func

## Bancos

### Extraindo a tabela **Bancos** da API

In [ ]:
# Extraindo a tabela banks da API
url = "https://brasilapi.com.br/api/banks/v1"

bancos = func.importar(url)

data = func.formato_datas()

bancos_json = func.salvar_tabela(bancos)

# Transformando as tabelas em um DataFrame
bancos_full = pd.DataFrame(bancos_json)

# Notificação 
notificacao.alerta(bancos, data, url)

### Tratamento dos dados do dataframe

In [ ]:
# Excluindo linhas com a coluna code que possuem missing values
bancos_full = bancos_full.dropna(subset=['code'])

#Transformando os dados da coluna code de float para int
bancos_full = bancos_full.astype({'code':int})

# Alterando o nome das colunas
bancos_full_renomeado = func.altera_colunas(bancos_full)

# Alterando a coluna Nome Completo para deixar todos nomes em maiusculo
bancos_full_renomeado['Nome Completo']=bancos_full_renomeado['Nome Completo'].str.upper()

#Tratamento de Missing Values
bancos_full_tratado = (bancos_full_renomeado.dropna())

# Ordenação pela coluna Código
bancos_full_tratado.sort_values('Código')


### Salvando os dataframes e dados brutos no Banco de Dados

In [ ]:
# Conectando com banco de dados e salvando os dados brutos
bancos_connect = sqlite3.connect('Projeto Final.db')
bancos_full.to_sql('Bancos Bruto', bancos_connect, if_exists='replace', index=False )

# Salvando os dados tratados no banco de dados e fechando a conexão
bancos_full_tratado.to_sql('Bancos Tratado', bancos_connect, if_exists='replace', index=False )
bancos_connect.close()

bancos_full_tratado

## Corretoras

### Extraindo a tabela **Corretoras** da API

In [ ]:
# Extraindo a tabela corretoras da API
url = "https://brasilapi.com.br/api/cvm/corretoras/v1"

corretoras = func.importar(url)

data = func.formato_datas()


corretoras_json = func.salvar_tabela(corretoras)

# Transformando as tabelas em um DataFrame
corretoras_full = pd.DataFrame(corretoras_json)

# Notificacao 

notificacao.alerta(corretoras, data, url)

### Tratamento dos dados do dataframe

In [ ]:
# Alterando o nome das colunas

corretoras_full_renomeado = func.nome_colunas(corretoras_full)

# Formatacao da coluna valor patrimonio

func.formato_valor(corretoras_full_renomeado,'Valor Patrimonio')

# Formatacao das colunas com datas

func.formato_data(corretoras_full_renomeado,'Data Registro')

# Exclusão de colunas irrelevantes para a análise. 

corretoras_full_colunas = func.exclusao_colunas(corretoras_full_renomeado)

# Ajustando o Formato do CNPJ

corretoras_full_colunas['CNPJ'] = func.formato_cnpj(corretoras_full_colunas,'CNPJ')

# Ajustando Formato do Telefone
filtro_tel =  func.filtro_str_len(corretoras_full_colunas,'Telefone',8) 
corretoras_full_colunas.loc[filtro_tel, 'Telefone'] = func.ajuste_telefone(corretoras_full_colunas,filtro_tel,'Telefone')
corretoras_full_colunas['Telefone'] = func.formato_telefone(corretoras_full_colunas, 'Telefone')

# Excluindo corretoras que não possuem Telefone na base de dados
filtro_sem_tel = func.string_vazia(corretoras_full_colunas,'Telefone',1) 
corretoras_full_colunas = func.drop_linhas(corretoras_full_colunas, filtro_sem_tel)

#Excluindo Corretoras que não possuem Pais na base de dados
filtro_pais = func.string_vazia(corretoras_full_colunas,'Pais',1)
corretoras_full_colunas = func.drop_linhas(corretoras_full_colunas,filtro_pais)

#Excluindo Corretoras que não possuem Municipio na base de dados
filtro_municipio = func.string_vazia(corretoras_full_colunas, 'Municipio', 0)
corretoras_full_colunas = func.drop_linhas(corretoras_full_colunas,filtro_municipio)

#Excluindo Corretoras que não possuem UF na base de dados
filtro_UF = func.string_vazia(corretoras_full_colunas,'UF',0)
corretoras_full_colunas = func.drop_linhas(corretoras_full_colunas,filtro_UF) 

#Excluindo Corretoras que não possuem Bairro na base de dados
filtro_bairro = func.string_vazia(corretoras_full_colunas,'Bairro',0)
corretoras_full_colunas = func.drop_linhas(corretoras_full_colunas,filtro_bairro)

#Excluindo corretoras que não possuem nome comercial
filtro_nome_comercial = func.string_vazia(corretoras_full_colunas,'Nome Comercial',0)
filtro_nome = func.caracter_especial(corretoras_full_colunas,'Nome Comercial','--')
corretoras_full_colunas = func.drop_linhas(corretoras_full_colunas,filtro_nome_comercial)
corretoras_full_colunas = func.drop_linhas(corretoras_full_colunas,filtro_nome)

#Excluindo corretoras que possuem valor de patrimonio zerado
filtro_valor = func.caracter_especial(corretoras_full_colunas,'Valor Patrimonio', 'R$0.00')
corretoras_full_colunas = func.drop_linhas(corretoras_full_colunas,filtro_valor)

# Filtrando as corretoras que estão em funcionamento
corretoras_ativas = corretoras_full_colunas.loc[corretoras_full_colunas['Status'] =='EM FUNCIONAMENTO NORMAL']

# Filtrando as corretoras que estão canceladas
corretoras_canceladas = corretoras_full_colunas.loc[corretoras_full_colunas['Status'] =='CANCELADA']

### Salvandos os dataframes e dados brutos no Banco de Dados

In [ ]:
# Conectando com banco de dados e salvando os dados brutos
corretoras_connect = sqlite3.connect('Projeto Final.db')
corretoras_full.to_sql('Corretoras Bruto', corretoras_connect, if_exists='replace', index=False )

# Salvando os dados tratados no banco de dados e fechando a conexão
corretoras_full_colunas.to_sql('Corretoras Tratado', corretoras_connect, if_exists='replace', index=False )
corretoras_ativas.to_sql('Corretoras Tratado Ativas', corretoras_connect, if_exists='replace', index=False )
corretoras_canceladas.to_sql('Corretoras Tratado Canceladas', corretoras_connect, if_exists='replace', index=False )
bancos_connect.close()

corretoras_full_colunas

## PIX

### Extraindo a tabela **PIX** da API

In [ ]:
# Extraindo a tabela PIX da API
url = 'https://brasilapi.com.br/api/pix/v1/participants'

pix = requests.get(url)

data = datetime.now().strftime("%d/%m/%Y %H:%M:%S")


pix_json = func.salvar_tabela(pix)

# Transformando as tabelas em um DataFrame
pix_full = pd.DataFrame(pix_json)

# Notificação
notificacao.alerta(pix, data, url)

### Tratamento dos dados do dataframe

In [ ]:
# Alterando o nome das colunas
pix_full_renomeado = func.modifica_coluna(pix_full)

# Tratamento da coluna Inicio que contem data e hora
pix_full_renomeado['Inicio'] = pd.to_datetime(pix_full_renomeado['Inicio'])
pix_full_renomeado['Inicio'] = func.ajusta_datahora(pix_full_renomeado,'Inicio')

### Salvando os dataframes e dados brutos no Banco de Dados

In [ ]:
# Conectando com banco de dados e salvando os dados brutos
pix_connect = sqlite3.connect('Projeto Final.db')
pix_full.to_sql('Pix Bruto', pix_connect, if_exists='replace', index=False )

# Salvando os dados tratados no banco de dados e fechando a conexão
pix_full_renomeado.to_sql('Pix Tratado', pix_connect, if_exists='replace', index=False )
pix_connect.close()


pix_full_renomeado